<a href="https://colab.research.google.com/github/Juanmbenitez00/QNTX/blob/main/notebooks_QNTX_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QNTX – Reportes Inteligentes para PyMEs  
**POC – Entrega Final (Fast Prompting)**  

Este notebook demuestra cómo transformar datos de ventas en:  

1. **Informe ejecutivo (Texto→Texto)**  
2. **Visualización conceptual (Texto→Imagen)**  

Se aplican técnicas *zero/one/few shot* y optimización de tokens (modelo económico, entrada mínima y límite de salida).  


In [8]:
import pandas as pd

# Si se utiliza API:
# from openai import OpenAI
# import os
# from dotenv import load_dotenv
# load_dotenv()
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [9]:
df = pd.read_csv("/content/sample_data/ventas_limpieza_julio.csv")
df["fecha"] = pd.to_datetime(df["fecha"])
df.head()


,fecha,id_producto,producto,unidades_vendidas,precio_unitario,ingresos,gasto_marketing,clientes_nuevos
0,2024-07-01,101,Detergente,11,1200,13200,16694,21
1,2024-07-01,102,Lavandina,16,800,12800,16694,21
2,2024-07-01,103,Escoba,64,2500,160000,16694,21
3,2024-07-01,104,Esponja,37,500,18500,16694,21
4,2024-07-01,105,Desinfectante,66,1500,99000,16694,21


In [10]:
# KPIs agregados por día
daily = df.groupby("fecha", as_index=False).agg(
    ingresos_totales=("ingresos","sum"),
    ordenes=("unidades_vendidas","sum"),
    gasto_marketing=("gasto_marketing","max"),  # mismo para todos los productos del día
    clientes_nuevos=("clientes_nuevos","max")   # mismo para todos los productos del día
)

# KPIs adicionales
daily["ticket_promedio"] = (daily["ingresos_totales"] / daily["ordenes"]).round(2)
daily["CAC"]  = (daily["gasto_marketing"] / daily["clientes_nuevos"]).round(2)
daily["ROAS"] = (daily["ingresos_totales"] / daily["gasto_marketing"]).round(2)
daily["crec_ingresos_%"] = (daily["ingresos_totales"].pct_change()*100).round(2)

daily.head(10)


,fecha,ingresos_totales,ordenes,gasto_marketing,clientes_nuevos,ticket_promedio,CAC,ROAS,crec_ingresos_%
0,2024-07-01,303500,194,16694,21,1564.43,794.95,18.18,NaN
1,2024-07-02,206400,181,8113,24,1140.33,338.04,25.44,-31.99
2,2024-07-03,380700,286,16254,7,1331.12,2322.00,23.42,84.45
3,2024-07-04,418500,292,17811,25,1433.22,712.44,23.50,9.93
4,2024-07-05,336200,269,13002,10,1249.81,1300.20,25.86,-19.67
5,2024-07-06,198900,161,15822,13,1235.40,1217.08,12.57,-40.84
6,2024-07-07,351100,231,8952,11,1519.91,813.82,39.22,76.52
7,2024-07-08,286600,206,17164,5,1391.26,3432.80,16.70,-18.37
8,2024-07-09,374800,282,14525,9,1329.08,1613.89,25.80,30.77
9,2024-07-10,293200,210,15058,21,1396.19,717.05,19.47,-21.77


In [11]:
# KPIs por producto (en todo julio)
por_producto = df.groupby(["id_producto","producto"], as_index=False).agg(
    unidades_totales=("unidades_vendidas","sum"),
    ingresos_totales=("ingresos","sum")
).sort_values("ingresos_totales", ascending=False)

# Producto más vendido (en unidades)
top_volumen = por_producto.sort_values("unidades_totales", ascending=False).head(1)

# Producto más rentable (en ingresos)
top_ingresos = por_producto.sort_values("ingresos_totales", ascending=False).head(1)

print("Producto más vendido (volumen):")
print(top_volumen)

print("\nProducto más rentable (ingresos):")
print(top_ingresos)


Producto más vendido (volumen):
   id_producto producto  unidades_totales  ingresos_totales
2          103   Escoba              1547           3867500

Producto más rentable (ingresos):
   id_producto producto  unidades_totales  ingresos_totales
2          103   Escoba              1547           3867500


In [12]:
# Resumen de los últimos 7 días de julio
ultimos7 = daily.tail(7).copy()

# Convertir a un formato resumido para enviar al modelo
resumen_texto = ultimos7.to_dict(orient="records")

ultimos7, resumen_texto


(        fecha  ingresos_totales  ordenes  gasto_marketing  clientes_nuevos  \
 24 2024-07-25            183400      136            18026               14   
 25 2024-07-26            284900      205            12307               25   
 26 2024-07-27            313700      237            13084               17   
 27 2024-07-28            298500      230            16669               21   
 28 2024-07-29            390700      312            14116                7   
 29 2024-07-30            359200      251            11588               13   
 30 2024-07-31            233100      211             8853               15   
 
     ticket_promedio      CAC   ROAS  crec_ingresos_%  
 24          1348.53  1287.57  10.17            11.35  
 25          1389.76   492.28  23.15            55.34  
 26          1323.63   769.65  23.98            10.11  
 27          1297.83   793.76  17.91            -4.85  
 28          1252.24  2016.57  27.68            30.89  
 29          1431.08   891.38 

In [13]:
# Prompt Texto → Texto
prompt_texto_texto = f"""
Actuá como analista de Business Intelligence senior.
Estos son los últimos 7 días de datos (valores por día): {resumen_texto}

Tareas:
1) Calculá/validá: crecimiento % de ingresos, CAC, ROAS y ticket promedio.
2) Identificá 3 tendencias o anomalías relevantes.
3) Redactá un informe ejecutivo (120–160 palabras): Qué pasó / Por qué / Qué hacer.
4) Proponé 5 acciones priorizadas (viñetas).

Formato de salida:
- Sección 1: Informe ejecutivo (párrafo).
- Sección 2: 3 bullets de tendencias/anomalías.
- Sección 3: 5 bullets de acciones.
"""

# Ejemplo de request (en este caso no se utiliza API KEY)
# response = client.responses.create(
#     model="gpt-4o-mini",
#     input=prompt_t2t,
#     temperature=0.3,
#     max_output_tokens=300
# )
# print(response.output_text)


In [15]:


# Pegar aquí la respuesta generada por ChatGPT Plus
informe_texto = """
Sección 1: Informe ejecutivo

En los últimos siete días se observaron fuertes oscilaciones en ingresos y eficiencia de marketing. Tras un pico de +55% el 26/7, los ingresos se desaceleraron y cerraron con una caída de –35% el 31/7. El CAC fue muy volátil (de $492 a más de $2.000), lo que evidencia desajustes en la captación de clientes. El ROAS se mantuvo alto (17–31), pero con caídas puntuales ligadas a menor efectividad de campañas. El ticket promedio mostró tendencia descendente, afectando la rentabilidad por cliente. La escoba se consolidó como producto líder en ventas e ingresos. En síntesis: se generaron buenos ingresos, pero con fuerte dependencia de campañas y falta de estabilidad en adquisición. Es clave estabilizar el costo de captación, revisar la estrategia de pricing y diversificar los drivers de ventas para no depender de un solo producto.

Sección 2: 3 tendencias/anomalías

Altísima volatilidad en ingresos: suba +55% (26/7) y desplome –35% (31/7).

CAC extremadamente variable: mínimo $492 (26/7) vs. máximo $2.016 (29/7).

Ticket promedio en descenso: de $1.389 (26/7) a $1.104 (31/7).

Sección 3: 5 acciones priorizadas

Revisar y segmentar campañas para estabilizar CAC y reducir picos de ineficiencia.

Implementar pricing dinámico o promociones cruzadas para sostener ticket promedio.

Diversificar catálogo y potenciar ventas de productos complementarios a la escoba.

Ajustar inversión en marketing hacia canales con mejor ROAS comprobado.

Crear alertas de monitoreo diario para ingresos, CAC y ticket promedio a fin de anticipar caídas.
"""

import os
os.makedirs("outputs", exist_ok=True)

# Guardar en Markdown
with open("outputs/reporte_QNTX.md", "w", encoding="utf-8") as f:
    f.write(informe_texto)

print("✔ Guardado en outputs/reporte_QNTX.md")


✔ Guardado en outputs/reporte_QNTX.md


In [18]:
## Prompt Texto → Imagen (ejemplo, no ejecutable)

#Se solicita al modelo generar un dashboard conceptual estilo corporativo, con fondo blanco y paleta azul/gris.
#El dashboard debe mostrar KPIs clave del último día (Ingresos, Órdenes, Ticket Promedio, CAC y ROAS).
#El objetivo es una visual conceptual para acompañar el informe, no un gráfico exacto.


In [ ]:
# === Prompt Texto → Imagen ===

# Tomar el último día de datos
ultimo = daily.iloc[-1]

# Prompt base
prompt_img = f"""
Diseñá un dashboard ejecutivo minimalista (1920x1080), fondo blanco, paleta azul/gris.
Título: "Resumen Diario – {ultimo['fecha'].date()}".

Mostrar tarjetas con:
- Ingresos: ${int(ultimo['ingresos_totales']):,}
- Órdenes: {int(ultimo['ordenes'])}
- Ticket promedio: ${ultimo['ticket_promedio']:.0f}
- CAC: ${ultimo['CAC']:.0f}
- ROAS: {ultimo['ROAS']:.2f}

Bordes suaves, sombras ligeras, estilo corporativo simple.
"""
print(prompt_img)

# Variante alternativa (ejemplo manual para generar la 2da imagen)
prompt_img_alt = f"""
Diseñá un dashboard ejecutivo minimalista (1920x1080), fondo blanco, paleta azul/gris.
Título: "Resumen Diario – {ultimo['fecha'].date()}".

Incluir:
- Tarjetas con KPIs (Ingresos, Órdenes, Ticket Promedio, CAC, ROAS).
- Un gráfico de barras con los últimos 7 días de ingresos.
- Estilo corporativo, minimalista, bordes suaves, legibilidad alta.
"""
print(prompt_img_alt)

"""
⚠ Nota:
Los prompts se ejecutaron en ChatGPT Plus (DALL·E) para obtener 2 imágenes distintas.
Se descargaron manualmente y se guardaron en:
- outputs/imagen_dashboard_1.png
- outputs/imagen_dashboard_2.png
"""


In [ ]:
## Comparativa de técnicas de Fast Prompting

- **Zero shot:** el modelo recibe el pedido directo (ej. *redactá un informe con KPIs*) sin ejemplos previos.

- **One shot:** se incluye un ejemplo de salida esperada (ej. *Sección 1: Informe / Sección 2: Tendencias / Sección 3: Acciones*).

- **Few shot:** se dan 2–3 ejemplos completos antes del pedido final.

En este proyecto se aplicó principalmente **one shot prompting** para guiar el formato del informe ejecutivo y garantizar salidas claras y útiles.


In [ ]:
## Conclusión

En este **Proyecto Final**, QNTX pasó de ser una idea a un flujo completo para generar reportes inteligentes en PyMEs.
Con un dataset de ventas se calcularon KPIs clave (Ingresos, CAC, ROAS, Ticket Promedio, Crecimiento) y se usaron técnicas de *prompting* para armar informes ejecutivos y visuales.

El resultado final incluye:
- Informe de texto con análisis y acciones.
- Visualizaciones tipo dashboard (2 variantes).
- Archivos exportados: `kpis_resumen.csv`, `reporte_QNTX.md` y las imágenes finales.

En resumen: una prueba clara de cómo la IA puede ayudar a las PyMEs a entender mejor sus datos y tomar decisiones más rápido.


